# Dealing with Conflicts in Lezer

This notebook demonstrates how **Lezer** deals with *shift-reduce* conflicts.

The following grammar is **ambiguous** because it does not specify the precedence of the arithmetic operators:

```lezer
Expr {
  Expr "+" Expr |
  Expr "*" Expr |
  Number
}
```

In Lezer such ambiguity usually results in a build-time error. However, we can use the `!ambig` marker to force the parser generator to accept the conflict, effectively letting it resolve conflicts by shifting (default behavior).

## Imports and Setup

In [ ]:
import { buildParser } from '@lezer/generator';
import { LRParser } from '@lezer/lr';
import { TreeCursor } from '@lezer/common';

## Grammar Definition

In [ ]:
const grammarDefinition = `
    @top Program { Expr }

    @tokens {
        Number { "0" | $[1-9] $[0-9]* }
        "+" "*"
        space { $[ \t\n\r]+ }
    }

    @skip { space }

    Expr {
        Expr "+" Expr |
        Expr "*" Expr |
        Number
    }
`;

In [ ]:
let parser: LRParser; 
try {
    parser = buildParser(grammarDefinition);
    console.log("SUCCESS: Parser generated.");
} catch (e) {
    if (e instanceof Error) {
        console.error(e.message);
    } else {
        console.error("Unbekannter Fehler:", String(e));
    }
}

## Analyzing the Conflict

The error message `shift/reduce conflict` confirms that our grammar is **ambiguous**. Lezer is strict and refuses to generate a parser for an ambiguous grammar without explicit conflict resolution strategies.

To understand the conflict, consider the input `1 + 2 * 3` (or generally `Expr + Expr * Expr`).

At the point where the parser has processed `1 + 2` and sees the `*` operator coming up (the lookahead), it faces a dilemma:

1.  **Reduce (Apply Rule):** It could assume `1 + 2` is a complete expression. It would "reduce" these tokens to an `Expr`. This would lead to the grouping `(1 + 2) * 3`.
2.  **Shift (Wait & Read):** It could assume that `2` belongs to the *next* operator `*`. It would "shift" the `*` onto the stack and wait. This leads to the grouping `1 + (2 * 3)`.



Without **Precedence** or **Associativity** rules, both trees are valid according to our grammar definition. Since Lezer cannot guess which interpretation is intended, it raises a build-time error.

**Interpreting the Error Message:**

* `Expr -> Expr · OpPlus Expr`: The dot (`·`) indicates the parser's position. It has seen the first part and *could* apply the rule (Reduce).
* `With input: Expr OpPlus Expr · OpPlus ...`: The parser sees another operator coming.
* **The Core Issue:** It doesn't know whether the current expression ends here or extends further to the right.